In [123]:
%load_ext autoreload
%autoreload 2

from pprint import pprint
import sys
import os, json, re, requests
from ckanapi import RemoteCKAN
import ckanapi.errors
from ckanapi.errors import NotFound, ValidationError
import pandas as pd
import random
import numpy as np

from pathlib import Path
import json

# from ckan_migration import Migrate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [131]:
ORIGINAL_CKAN_URL = 'https://staging.basedosdados.org'
# ORIGINAL_CKAN_URL = 'http://localhost:5000'

In [132]:
def download_packages():
    api_url = ORIGINAL_CKAN_URL + '/api/3/action/package_search?q=&rows=2000'
    packages = requests.get(api_url, verify=False).json()['result']['results']
    for p in packages:
        if not os.path.isdir("packages/"):
            os.mkdir('packages/')
        name= p['name']
        json.dump(p, open(f'packages/{name}', 'w'))
    return packages


def ckan_api_staging():
#     user_agent = None
    CKAN_API_KEY = "api_key"
    CKAN_URL = os.environ.get("CKAN_URL", ORIGINAL_CKAN_URL)

    return RemoteCKAN(CKAN_URL, apikey=CKAN_API_KEY)

def update_staging(package_dict):
    try:
        ckan_api_staging().action.package_update(**package_dict)
    except NotFound as e:
        print(e)


def validate(package_dict):
        try:
            return ckan_api_staging().action.package_validate(**package_dict)
        except ValidationError as e:
            return e.error_dict["message"]    

In [134]:
# all_packages = download_packages()

In [135]:
all_packages_dict = {p['name']:p for p in all_packages}

null_titles_dash = {}
null_titles_underscore = {}

for package_name in all_packages_dict:
    package = all_packages_dict[package_name]
    if not package['title']:
        null_titles_dash[package['name']]=package
        null_titles_underscore[package['name'].replace('-','_')] = all_packages_dict.get(package['name'].replace('-','_'))

In [136]:
### Packages com title null
list(null_titles_dash.keys())

['mundo-transrespect-transphobia',
 'eu-fra-lgbt',
 'br-ponte-indicadores',
 'br-me-cno',
 'br-mc-indicadores',
 'br-isp-estatisticas-seguranca',
 'br-inmet-bdmep',
 'br-ibge-cbo-2002',
 'br-ggb-relatorio-lgbtqi',
 'br-cgu-servidores-executivo-federal',
 'br-cgu-pessoal-executivo-federal',
 'br-camara-dados-abertos',
 'br-camara-atividade-legislativa',
 'br-abrinq-oca']

In [137]:
### Packages duplicados
[title for title in null_titles_underscore if null_titles_underscore[title] is not None]

['br_camara_atividade_legislativa', 'br_abrinq_oca']